# NIH Grant Spreadsheet Validator

## Methodology

Steps:
1. Import csv with data to be checked
2. Clean, flatten, and join (if necessary) unvalidated data
    1. Check for duplications 
3. Use unique identifier to pull desired fields from NIH Reporter
4. Validate
    1. Check if any unique identifiers did not find grants
    2. Check key values while noting any errors
5. Output results to excel 



## Data Load

### Load data to validate

In [549]:
import requests
import pandas as pd
import numpy as np
import math
import re

In [550]:
# TODO: Import spreadsheet to validate
df_to_validate = pd.read_csv("./data/grant_data.csv")
# df_to_validate = pd.read_csv("./data/your_data_dirty.csv")
df_to_validate.head()

,For Matching,PI Name,Ref Awd #,Sponsor,Proj Start,Proj End,Budget Start,Budget End,Direct,Indirect,Is Cancer Relevant,Is Multi-PI,PI List,Project Title
0,1,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$60,178.42","$35,304.13",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
1,2,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$31,307.90","$18,158.94",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
2,3,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$11,555.65","$6,798.00",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
3,4,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$13,888.48","$8,054.76",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
4,5,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$13,140.37","$7,620.58",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...


### Reformat data to validate

In [551]:
# Create a dictionary to translate column names to the API's expected format
column_translation = {
    'Ref Awd #': 'project_num',
    'PROJECT_NUM': 'project_num',
    'ORGANIZATION': 'organization',
    'AWARD_AMOUNT': 'award_amount',
    'PI Name': 'principal_investigators',
    'Sponsor': 'agency_ic_admin.name',
    'AGENCY_IC_FUNDINGS': 'agency_ic_fundings',
    'Proj Start': 'project_start_date',
    'Proj End': 'project_end_date',
    'Project Title': 'project_title',
    'Budget Start': 'budget_start',
    'Budget End': 'budget_end',
    ' Direct ': 'direct_cost_amt',
    ' Indirect ': 'indirect_cost_amt',
    # Add more translations as needed
}

# Update the DataFrame to match the API's expected column names
df_to_validate.rename(columns=column_translation, inplace=True)

In [552]:
df_to_validate

,For Matching,principal_investigators,project_num,agency_ic_admin.name,project_start_date,project_end_date,budget_start,budget_end,direct_cost_amt,indirect_cost_amt,Is Cancer Relevant,Is Multi-PI,PI List,project_title
0,1,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$60,178.42","$35,304.13",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
1,2,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$31,307.90","$18,158.94",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
2,3,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$11,555.65","$6,798.00",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
3,4,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$13,888.48","$8,054.76",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
4,5,"Tsao, Anne",5U10CA180858-05,National Cancer Institute,4/15/2014,8/31/2020,3/1/2018,8/31/2020,"$13,140.37","$7,620.58",Yes - Cancer Only Sponsor,Yes,Hunt K\nPinnix C\nTsao A,UT MD Anderson Cancer Center Network Lead Acad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,895,"Shetty, Gunapala",5P01HD075795-05,MAGEE-WOMEN'S RES INST AND FOUNDATION,7/1/2014,6/30/2020,7/1/2018,6/30/2020,"$154,529.00","$92,718.00","Yes - kw/acro (3/2 [CANCER,CHEMOTHER,RADIATION...",NaN,Shetty G,Preserving Male Fertility after Cancer Therapy...
895,896,"Bresalier, Robert",5U24CA086368-19,FRED HUTCHINSON CANCER RESEARCH CENTER,4/1/2018,3/31/2022,4/1/2019,3/31/2022,"$46,587.00","$27,952.00",Yes - Cancer Only Sponsor,NaN,Bresalier R,Steps Towards Validation of Plasma Biomarkers ...
896,897,"Friedl, Peter",5U54CA210184-04,CORNELL UNIVERSITY,8/29/2016,7/31/2021,8/1/2019,7/31/2020,"$75,000.00","$45,000.00",Yes - Cancer Only Primary Sponsor,Yes,Friedl P,Center on They Physics of Cancer Metabolism
897,898,"Friedl, Peter",5U54CA210184-03,CORNELL UNIVERSITY,8/29/2016,7/31/2021,8/1/2018,7/31/2021,"$95,000.00","$57,000.00",Yes - Cancer Only Primary Sponsor,Yes,Friedl P,Supplemental Funding: Center on the Physics of...


In [553]:
# Reformat project_num column in df_to_validate to uppercase and remove leading/trailing whitespace
df_to_validate['project_num'] = df_to_validate['project_num'].str.upper().str.strip()

In [554]:
# Create copy of the validation DataFrame to place edited info and add comments to
df_validation_wip_output = df_to_validate.copy()
df_validation_wip_output['comments'] = ""  # Add a comments column for manual review

In [555]:
# Find rows with missing project_num and add a comment
num_missing = 0
if 'project_num' in df_validation_wip_output.columns:  
    num_missing = df_validation_wip_output['project_num'].isnull().sum()
    if num_missing > 0:
        missing_projects = df_validation_wip_output[df_validation_wip_output['project_num'].isnull()]
        for index, row in missing_projects.iterrows():
            df_validation_wip_output.at[index, 'comments'] += "Missing project_num\n"
print(f"Number of missing project_num entries: {num_missing}")

Number of missing project_num entries: 1


In [556]:
# Check for duplicate project_ids
num_duplicate_projects = df_to_validate.duplicated(subset=['project_num']).sum()
print(f"Number of duplicates based on project number: {num_duplicate_projects}")

Number of duplicates based on project number: 225


In [557]:
# Build unique list of project_ids to validate
unique_project_ids = df_to_validate['project_num'].unique()

# Remove nan values from the list
unique_project_ids = unique_project_ids[~pd.isnull(unique_project_ids)].tolist()

# Ensure all project_ids are strings
unique_project_ids = [str(pid).strip() for pid in unique_project_ids if pd.notnull(pid)]

# Remove empty strings or whitespace-only entries
unique_project_ids = [pid for pid in unique_project_ids if pid != ""]

### Validate project ids

In [558]:
def is_valid_project_num(pid):
    """
    Returns True if pid matches NIH project number pattern.
    Example valid: 5R01CA123456-01, 3R01HL098765-04S1
    """
    pattern = r"^[0-9][A-Z][A-Z0-9]{2}[A-Z]{2}[0-9]{6}(-[0-9]{2}[A-Z0-9]*)?$"
    return bool(re.match(pattern, pid.strip())) if isinstance(pid, str) else False

def validate_project_ids(project_id_list, log_invalid=False, log_filename="invalid_ids.txt"):
    """
    Validates NIH project IDs, returns list of valid ones.
    
    Parameters:
        project_id_list (list): list of project IDs (strings)
        log_invalid (bool): if True, saves invalid IDs to a file
        log_filename (str): file path to save invalid IDs (if enabled)

    Returns:
        valid_ids (list): list of validated project numbers
    """
    valid_ids = []
    invalid_ids = []

    for pid in project_id_list:
        if is_valid_project_num(pid):
            valid_ids.append(pid.strip())
        else:
            invalid_ids.append(pid)

    print(f"✅ {len(valid_ids)} valid project IDs")
    print(f"⚠️  {len(invalid_ids)} invalid project IDs")

    if log_invalid and invalid_ids:
        with open(log_filename, "w") as f:
            for pid in invalid_ids:
                f.write(str(pid) + "\n")
        print(f"📄 Invalid IDs written to {log_filename}")

    return valid_ids

In [559]:
# Validate
clean_ids = validate_project_ids(unique_project_ids, log_invalid=True)

# # Now safely call your NIH API function
# df = fetch_nih_projects(clean_ids)

✅ 235 valid project IDs
⚠️  438 invalid project IDs
📄 Invalid IDs written to invalid_ids.txt


In [560]:
# Add comments to the end for rows with invalid project numbers. If a comment already exists for this row, then append to end with a comma
df_validation_wip_output['comments'] = df_validation_wip_output['comments'].fillna('')

if 'project_num' in df_validation_wip_output.columns:
    for index, row in df_validation_wip_output.iterrows():
        if not is_valid_project_num(row['project_num']):
            if row['comments']:
                df_validation_wip_output.at[index, 'comments'] += ", Invalid project number"
            else:
                df_validation_wip_output.at[index, 'comments'] = "Invalid project number"



In [561]:
# # Add or append comments for rows with invalid project numbers
# for index, row in df_validation_wip_output.iterrows():
#     if row['project_num'] not in clean_ids:
#         df_validation_wip_output.at[index, 'comments'] += "Invalid project_num\n"



### Pull data from NIH RePORTER

In [562]:
# def fetch_nih_projects(search_text="cancer", fiscal_years=[2023], max_results=1000, limit=500, unique_project_ids=None):
def fetch_nih_projects(max_results=1000, limit=500, unique_project_ids=None):
    """
    Fetch project data from NIH RePORTER API up to a maximum number of results.

    Parameters:
        search_text (str): Text to search for in project descriptions.
        fiscal_years (list): List of fiscal years to filter results.
        max_results (int): Maximum number of results to return.
        limit (int): Number of results per request (max is 500).

    Returns:
        DataFrame of results
    """
    url = "https://api.reporter.nih.gov/v2/projects/search"
    offset = 0
    all_results = []

    while offset < max_results:
        payload = {
            "criteria": {
                "project_nums": unique_project_ids,
                # "text_search": {
                #     "search_text": search_text
                # }
            },
            "include_fields": [
                # "ApplId",
                "SubprojectId",
                # "FiscalYear",
                "ProjectNum",
                "ProjectSerialNum",
                # "Organization", 
                # "OrganizationType", 
                # "AwardType", 
                # "ActivityCode", 
                # "AwardAmount", 
                # "ProjectNumSplit", 
                "PrincipalInvestigators", 
                # "ProgramOfficers", 
                "AgencyIcAdmin", 
                # "AgencyIcFundings",
                # "CongDist", 
                "ProjectStartDate",
                "ProjectEndDate",
                # "FullFoa",
                # "FullStudySection",
                # "AwardNoticeDate", 
                # "CoreProjectNum",
                # "PrefTerms", 
                "ProjectTitle", 
                # "PhrText",
                # "SpendingCategoriesDesc", 
                # "ArraFunded",
                "BudgetStart", 
                "BudgetEnd",
                # "CfdaCode",
                # "FundingMechanism",
                "DirectCostAmt",
                "IndirectCostAmt" 
            ],
            "offset": offset,
            "limit": min(limit, max_results - offset)  # Prevent over-requesting
        }

        response = requests.post(url, json=payload)
        if response.status_code != 200:
            print("Request failed:", response.status_code)
            break

        results = response.json().get("results", [])
        if not results:
            break  # No more results

        all_results.extend(results)
        offset += len(results)

    df = pd.DataFrame(all_results)
    print(f"✅ Retrieved {len(df)} total records.")
    return df

In [563]:
# Fetch NIH projects from the API using the validated project IDs
df_nih = fetch_nih_projects(unique_project_ids=clean_ids)

✅ Retrieved 328 total records.


In [564]:
# Preview
df_nih.head()

,subproject_id,project_num,project_serial_num,principal_investigators,agency_ic_admin,project_start_date,project_end_date,project_title,budget_start,budget_end,direct_cost_amt,indirect_cost_amt
0,None,5R01CA047296-32,CA047296,"[{'profile_id': 7748957, 'first_name': 'GUILLE...","{'code': 'CA', 'abbreviation': 'NCI', 'name': ...",1988-07-01T00:00:00,2021-11-30T00:00:00,A Pathway of Tumor Suppression,2019-12-01T00:00:00,2020-11-30T00:00:00,228750,137250.0
1,None,1R01CA236864-01A1,CA236864,"[{'profile_id': 10771205, 'first_name': 'Nicho...","{'code': 'CA', 'abbreviation': 'NCI', 'name': ...",2019-12-01T00:00:00,2024-11-30T00:00:00,DELINEATING THE EVOLUTION AND ECOLOGY OF CHEMO...,2019-12-01T00:00:00,2020-11-30T00:00:00,383071,237504.0
2,None,4R37HD030284-26,HD030284,"[{'profile_id': 1896260, 'first_name': 'Richar...","{'code': 'HD', 'abbreviation': 'NICHD', 'name'...",1994-01-01T00:00:00,2024-05-31T00:00:00,Female Reproductive Tract Development in the m...,2019-06-01T00:00:00,2020-05-31T00:00:00,250000,150000.0
3,None,5R21CA218543-02,CA218543,"[{'profile_id': 10713323, 'first_name': 'Eliza...","{'code': 'CA', 'abbreviation': 'NCI', 'name': ...",2018-07-01T00:00:00,2021-06-30T00:00:00,Narrative visualization for breast cancer surv...,2019-07-01T00:00:00,2021-06-30T00:00:00,114374,52592.0
4,None,5R21CA220299-02,CA220299,"[{'profile_id': 9060637, 'first_name': 'Veerab...","{'code': 'CA', 'abbreviation': 'NCI', 'name': ...",2018-06-08T00:00:00,2021-05-31T00:00:00,Proteomic-based integrated subject-specific ne...,2019-06-01T00:00:00,2021-05-31T00:00:00,105487,63292.0


### Reformat NIH data

In [565]:
def flatten_nested_columns(df, 
                            prefix_sep='.', 
                            drop_original=True, 
                            list_handling='first',  # 'first', 'combine', 'explode'
                            list_combine_sep=' | ',
                            verbose=False):
    """
    Flattens nested dictionary and list-of-dict columns in a DataFrame.

    Parameters:
        df (pd.DataFrame): Input DataFrame
        prefix_sep (str): Separator for flattened nested keys
        drop_original (bool): Drop original nested columns after flattening
        list_handling (str): How to handle list-of-dict columns:
                             - 'first': Keep first item only
                             - 'combine': Join values into one string
                             - 'explode': One row per item
        list_combine_sep (str): Separator used in 'combine' mode
        verbose (bool): If True, print each column being processed

    Returns:
        pd.DataFrame: Flattened DataFrame
    """
    df = df.copy()
    
    for col in df.columns:
        if verbose:
            print(f"Processing column: {col}")

        # Flatten dictionary columns
        if df[col].apply(lambda x: isinstance(x, dict)).any():
            expanded = df[col].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series)
            expanded.columns = [f"{col}{prefix_sep}{sub}" for sub in expanded.columns]
            if drop_original:
                df = df.drop(columns=[col])
            df = pd.concat([df, expanded], axis=1)

        # Handle list of dicts
        elif df[col].apply(lambda x: isinstance(x, list) and all(isinstance(i, dict) for i in x)).any():

            if list_handling == 'first':
                # Use only first dict in the list
                expanded = df[col].apply(lambda x: x[0] if isinstance(x, list) and x else {}).apply(pd.Series)
                expanded.columns = [f"{col}{prefix_sep}{sub}" for sub in expanded.columns]
                if drop_original:
                    df = df.drop(columns=[col])
                df = pd.concat([df, expanded], axis=1)

            elif list_handling == 'combine':
                def combine_dicts(lst):
                    if isinstance(lst, list):
                        return list_combine_sep.join([
                            ' '.join(str(v) for v in d.values() if v) for d in lst
                        ])
                    return None
                df[f"{col}_combined"] = df[col].apply(combine_dicts)
                if drop_original:
                    df = df.drop(columns=[col])

            elif list_handling == 'explode':
                df = df.explode(col).reset_index(drop=True)
                expanded = df[col].apply(lambda x: x if isinstance(x, dict) else {}).apply(pd.Series)
                expanded.columns = [f"{col}{prefix_sep}{sub}" for sub in expanded.columns]
                if drop_original:
                    df = df.drop(columns=[col])
                df = pd.concat([df, expanded], axis=1)

    return df


In [566]:
df_nih_flat = flatten_nested_columns(df_nih)
df_nih_flat.head()

,subproject_id,project_num,project_serial_num,project_start_date,project_end_date,project_title,budget_start,budget_end,direct_cost_amt,indirect_cost_amt,principal_investigators.profile_id,principal_investigators.first_name,principal_investigators.middle_name,principal_investigators.last_name,principal_investigators.is_contact_pi,principal_investigators.full_name,principal_investigators.title,agency_ic_admin.code,agency_ic_admin.abbreviation,agency_ic_admin.name
0,None,5R01CA047296-32,CA047296,1988-07-01T00:00:00,2021-11-30T00:00:00,A Pathway of Tumor Suppression,2019-12-01T00:00:00,2020-11-30T00:00:00,228750,137250.0,7748957,GUILLERMINA,,LOZANO,True,GUILLERMINA LOZANO,,CA,NCI,National Cancer Institute
1,None,1R01CA236864-01A1,CA236864,2019-12-01T00:00:00,2024-11-30T00:00:00,DELINEATING THE EVOLUTION AND ECOLOGY OF CHEMO...,2019-12-01T00:00:00,2020-11-30T00:00:00,383071,237504.0,10771205,Nicholas,,Navin,True,Nicholas Navin,,CA,NCI,National Cancer Institute
2,None,4R37HD030284-26,HD030284,1994-01-01T00:00:00,2024-05-31T00:00:00,Female Reproductive Tract Development in the m...,2019-06-01T00:00:00,2020-05-31T00:00:00,250000,150000.0,1896260,Richard,R,Behringer,True,Richard R Behringer,PROFESSOR AND DEPUTY CHAIR,HD,NICHD,Eunice Kennedy Shriver National Institute of C...
3,None,5R21CA218543-02,CA218543,2018-07-01T00:00:00,2021-06-30T00:00:00,Narrative visualization for breast cancer surv...,2019-07-01T00:00:00,2021-06-30T00:00:00,114374,52592.0,10713323,Elizabeth,J.,Lyons,True,Elizabeth J. Lyons,ASSOCIATE PROFESSOR,CA,NCI,National Cancer Institute
4,None,5R21CA220299-02,CA220299,2018-06-08T00:00:00,2021-05-31T00:00:00,Proteomic-based integrated subject-specific ne...,2019-06-01T00:00:00,2021-05-31T00:00:00,105487,63292.0,9060637,Veerabhadran,,Baladandayuthapani,False,Veerabhadran Baladandayuthapani,,CA,NCI,National Cancer Institute


## Data Validation

### Project ID
Find missing project IDs

In [567]:
# Check for missing project_ids from the API response
if 'project_num' in df_nih_flat.columns:
    missing_project_ids = set(unique_project_ids) - set(df_nih_flat['project_num'].dropna().unique())
    if missing_project_ids:
        print(f"Missing project IDs from API response: {missing_project_ids}\n Number of missing IDs: {len(missing_project_ids)}")
    else:
        print("All requested project IDs were found in the API response.")



Missing project IDs from API response: {'FP00002252', 'RP190599', 'W81XWH-17-1-0293', '2018-0065', '19-40-41-JUNG', '2019-0034', '2016-0838', 'HHSN261201200034I-TO3', 'W81XWH-17-1-0465', '2018-0027', '16052989', 'CCR14300865', 'SU2C-AACR-IRG-19-17', 'RP170079', '10000001932', '2016-0432', '2014-0619', '5U10CA086400-19', 'LS2019-00057038-JW', 'PA18-0924', '2017-0840', 'C7636/A25272', '55097', '2016-0332', 'FP00001919', '2013-0957', 'LS2019-00056780-LG', '2018-0486', '2018-1112', '2018-0946', 'FP00004919', 'PA19-0033', '2018-0253', '2014-0221', 'FP00000614', '2018-1182', 'HHSN261201200034I-TO9', '1926306', 'HHSN261201500018I', 'W81XWH-16-1-0027', '2013-0258', 'PP150054', 'PF-19-028-01-DMC', 'DAA-17-63736-0', '2016-0537', '2017-0361', '2019-0274', 'FP00007352', '2018-0797', 'LAPS-TX035', '2013-0114', '2017-0330', '2T15LM007093-27', 'PDF17487910', '2019-0568', '2017-0988', '2018-0576', '2016-0527', 'RP180463', 'W81XWH-19-1-0410', '2019-0149', '2017-0820', 'IIP-1823712', 'RP180505', '2019-0

In [568]:
if 'project_num' in df_nih_flat.columns:
    missing_projects = set(unique_project_ids) - set(df_nih_flat['project_num'].dropna().unique())
    
    for pid in missing_projects:
        if pid in df_validation_wip_output['project_num'].values:
            condition = df_validation_wip_output['project_num'] == pid
            
            # Get current comments
            current_comments = df_validation_wip_output.loc[condition, 'comments'].fillna('')
            
            # Create new comments by appending with comma if needed
            new_comments = current_comments.apply(
                lambda x: x + f", Project_Id missing from API response: {pid}" if x else f"Project_Id missing from API response: {pid}"
            )

            # Assign back
            df_validation_wip_output.loc[condition, 'comments'] = new_comments
        else:
            print(f"Project ID {pid} not found in validation DataFrame.")


In [569]:
# Add flag to indicate if project_num was found in the API response
if 'project_num' in df_nih_flat.columns:
    df_validation_wip_output['found_in_api'] = df_validation_wip_output['project_num'].isin(df_nih_flat['project_num'])

### Project Title

In [570]:
from rapidfuzz import fuzz, process

def fuzzy_match_by_column(
    df_to_validate,
    df_reference,
    key_column,           # shared column (e.g., project_num)
    match_column,         # column to validate (e.g., project_title)
    reference_column,     # correct values (e.g., project_title)
    comment_column='comments',
    threshold=85,
    suffix=""
):
    """
    General-purpose fuzzy matcher between two dataframes using a shared key and string columns.
    Adds best match, score, possible matches, and updates comments.

    Parameters:
        df_to_validate: DataFrame with messy data
        df_reference: DataFrame with valid data
        key_column: column shared by both (e.g. project_num)
        match_column: text column to validate (e.g. project_title)
        reference_column: reference text column to match against
        comment_column: optional column to append match feedback
        threshold: minimum score for fuzzy match
        suffix: string to append to result column names (e.g. "_title")

    Returns:
        Updated df with:
            - best_match{suffix}
            - similarity_score{suffix}
            - possible_matches{suffix}
            - updated comments
    """
    best_matches = []
    match_scores = []
    updated_comments = []
    possible_matches_list = []

    for _, row in df_to_validate.iterrows():
        key_val = row[key_column]
        value_to_check = str(row[match_column]).strip()
        existing_comment = str(row.get(comment_column, '')).strip()

        # Get candidate values for the matching key
        candidates = df_reference[df_reference[key_column] == key_val][reference_column].dropna().unique().tolist()

        match_result = None
        score = None
        new_comment = None
        possible_matches = []

        if not candidates:
            new_comment = f'No {suffix} matched closely'
        elif value_to_check.lower() in [c.lower() for c in candidates]:
            # Exact match — no changes
            new_comment = None
        else:
            # Map lowercase → original for safe matching
            lower_to_original = {c.lower(): c for c in candidates}

            match_lc, score, _ = process.extractOne(
                value_to_check.lower(),
                list(lower_to_original.keys()),
                scorer=fuzz.token_sort_ratio
            )

            match_result = lower_to_original.get(match_lc)

            if score >= threshold:
                new_comment = f'{suffix} updated'
            else:
                new_comment = f'No {suffix} matched closely'
                match_result = None
                possible_matches = candidates

        # Combine comment
        if new_comment:
            if existing_comment and new_comment not in existing_comment:
                updated_comment = existing_comment + ", " + new_comment
            elif existing_comment:
                updated_comment = existing_comment
            else:
                updated_comment = new_comment
        else:
            updated_comment = existing_comment

        best_matches.append(match_result)
        match_scores.append(score)
        updated_comments.append(updated_comment)
        possible_matches_list.append(possible_matches)

    # Column names with suffix
    match_col = f'best_match_{suffix}'
    score_col = f'similarity_score_{suffix}'
    possible_col = f'possible_matches_{suffix}'

    df_out = df_to_validate.copy()
    df_out[match_col] = best_matches
    df_out[score_col] = match_scores
    df_out[possible_col] = possible_matches_list
    df_out[comment_column] = updated_comments

    return df_out


In [571]:
# Perform fuzzy matching on project titles
df_validation_wip_output = fuzzy_match_by_column(
    df_to_validate=df_validation_wip_output,
    df_reference=df_nih_flat,
    key_column='project_num',
    match_column='project_title',
    reference_column='project_title',
    comment_column='comments',
    threshold=55,
    suffix="title"
)


### PI Name

### Agency IC Name

In [572]:
# Perform fuzzy matching on project titles
df_validation_wip_output = fuzzy_match_by_column(
    df_to_validate=df_validation_wip_output,
    df_reference=df_nih_flat,
    key_column='project_num',
    match_column='agency_ic_admin.name',
    reference_column='agency_ic_admin.name',
    comment_column='comments',
    threshold=55,
    suffix="sponsor"
)

### Dates

In [573]:
# Convert time columns to datetime
time_columns = ['project_start_date', 'project_end_date', 'budget_start', 'budget_end']
for col in time_columns:   
    if col in df_validation_wip_output.columns:
        df_validation_wip_output[col] = pd.to_datetime(df_validation_wip_output[col], errors='coerce')
# Save the updated DataFrame to a new CSV file


#### Budget Dates

In [574]:
# Compare budget start and end dates
if 'budget_start' in df_validation_wip_output.columns and 'budget_end' in df_validation_wip_output.columns:
    budget_start = df_validation_wip_output['budget_start']
    budget_end = df_validation_wip_output['budget_end']

    # Check if budget start is after budget end
    invalid_budgets = budget_start > budget_end

    if invalid_budgets.any():
        for index in df_validation_wip_output[invalid_budgets].index:
            current_comments = df_validation_wip_output.at[index, 'comments']
            new_comment = "Budget start date is after budget end date"
            if current_comments:
                df_validation_wip_output.at[index, 'comments'] += f", {new_comment}"
            else:
                df_validation_wip_output.at[index, 'comments'] = new_comment
    
    # Check if budget start dates match df_flat project start dates and create budget_start_difference column
    if 'budget_start' in df_validation_wip_output.columns:   
        budget_start_difference = (budget_start - df_validation_wip_output['budget_start']).dt.days
        df_validation_wip_output['budget_start_difference'] = budget_start_difference

        # Add comments for mismatches
        mismatched_starts = budget_start != df_validation_wip_output['budget_start']
        if mismatched_starts.any():
            for index in df_validation_wip_output[mismatched_starts].index:
                current_comments = df_validation_wip_output.at[index, 'comments']
                new_comment = "Budget start date does not match project start date"
                if current_comments:
                    df_validation_wip_output.at[index, 'comments'] += f", {new_comment}"
                else:
                    df_validation_wip_output.at[index, 'comments'] = new_comment

                    

#### Project Dates

### Costs

#### Direct

#### Indirect 

In [575]:
#### Indirect

In [576]:
# Re